In [ ]:
import numpy as np
import os
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.layers.normalization import BatchNormalization
import keras.utils
from keras import layers , activations
from keras.optimizers import SGD

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
# model.fit 단계에서 validation_split 옵션 사용예정 

# 미사용 로직
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000 ]

In [ ]:
import keras.utils

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_valid = keras.utils.to_categorical(y_valid, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
# conv_base=tf.keras.applications.resnet50.ResNet50(
#                   weights='imagenet',  ## 모델을 포기화할 checkpoint 지정 (여기는 imagenet 학습 모델)
#                   include_top=False   ## 네트워크 최상위 FC(분류기)를 포함할지 안할지 지정 (imagenet은 1000개 클래스인데, 우리는 10개 클래스만 사용할 예정이므로)
#                   )
conv_base = tf.keras.applications.vgg16.VGG16(weights='imagenet',  ## 모델을 포기화할 checkpoint 지정 (여기는 imagenet 학습 모델)
                  include_top=False )

conv_base.summary()

# for layer in conv_base.layers: layer.trainable=False

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [ ]:
conv_base.trainable = True

print(conv_base.layers)
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':  ## block5_conv1이후의 모든 레이어들은 학습이 된다.
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

[<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd8404609e8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd87391bdd8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd87391d0b8>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fd87391d5f8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd87391e470>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd83f1c5710>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fd83f1c5b00>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd8301496a0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd830151518>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd830151940>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fd83015a518>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd83015acf8>, <tensorflow.python.keras.layers.convo

In [ ]:
Input_Layer = tf.keras.layers.Input(shape=(32,32,3))
x=conv_base(Input_Layer)
x=tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(512, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=[Input_Layer], outputs=[Out_Layer])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 14,982,474
Trainable params: 7,347,210
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=0.0001)

model.compile(optimizer= optimizer, loss ='categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), 
                    steps_per_epoch=x_train.shape[0] // batch_size, 
                    epochs=40, 
                    validation_data=(x_valid,y_valid))

Epoch 1/40
450/450 [==============================] - 23s 52ms/step - loss: 1.7625 - accuracy: 0.3793 - val_loss: 1.1665 - val_accuracy: 0.5826
Epoch 2/40
450/450 [==============================] - 23s 51ms/step - loss: 1.0786 - accuracy: 0.6184 - val_loss: 0.9260 - val_accuracy: 0.6882
Epoch 3/40
450/450 [==============================] - 24s 53ms/step - loss: 0.9098 - accuracy: 0.6878 - val_loss: 0.8449 - val_accuracy: 0.7204
Epoch 4/40
450/450 [==============================] - 23s 51ms/step - loss: 0.8052 - accuracy: 0.7209 - val_loss: 0.7647 - val_accuracy: 0.7482
Epoch 5/40
450/450 [==============================] - 23s 51ms/step - loss: 0.7499 - accuracy: 0.7423 - val_loss: 0.7097 - val_accuracy: 0.7694
Epoch 6/40
450/450 [==============================] - 23s 52ms/step - loss: 0.6940 - accuracy: 0.7613 - val_loss: 0.7265 - val_accuracy: 0.7650
Epoch 7/40
450/450 [==============================] - 23s 50ms/step - loss: 0.6551 - accuracy: 0.7731 - val_loss: 0.7045 - val_accuracy:

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=256)

40/40 [==============================] - 1s 27ms/step - loss: 1.0965 - accuracy: 0.7731
